In [30]:
import numpy as np
from numpy import log
import matplotlib.pyplot as plt  

import scipy.optimize as optimize

import scipy.io
import random

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier

In [20]:
def addOnes(X):
    m = X.shape[0]
    ones = np.ones(m)
    onesX = np.column_stack((ones, X))
    return onesX

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def initTheta(n, option=0):
    theta= np.zeros(n)
    if option != 0:
        theta = np.random.random(n)
    return theta.reshape(-1,1)

def computeCostLog(theta, X, y, lamda = 0.):
    X = addOnes(X)
    m = X.shape[0]
    y = y.reshape(-1,1)
    
    theta = theta.reshape(-1,1)
    theta0 = np.copy(theta); theta[0] = 0;
    
    h = sigmoid(X @ theta)
    J =  - y.T @ log(h) - (1- y).T @ log(1-h) + (lamda / 2) * theta0.T @ theta0
    J = J / (1.0 * m)
    return J[0,0]

def computeGradLog(theta, X, y, lamda = 0.):
    X = addOnes(X)
    m = X.shape[0]
    y = y.reshape(-1,1)
    
    theta = theta.reshape(-1,1)
    theta0 = np.copy(theta); theta[0] = 0;
    
    h = sigmoid(X @ theta)
    grad =  (X.T @ (h - y)) + lamda * theta0
    grad = grad / (1.0 * m)
    grad = grad.ravel()
    return grad

def gradientDescentLog(X, y,lamda = 0., alpha = 0.01, numIter = 1500, seed = 0):
    X = addOnes(X)
    m = X.shape[0]
    n = X.shape[1]
    y = y.reshape(-1,1)
    theta = initTheta(n, seed)
    theta0 = np.copy(theta); theta[0] = 0;
    
    J = np.zeros(numIter)
    for i in range(0, numIter):
        h = sigmoid(X @ theta)
        cost =  (1.0/m) * ((- y.T @ log(h) - (1- y).T @ log(1-h)) + (lamda / 2) * theta0.T @ theta0 )
        J[i] = cost[0]

        gradient =  (1.0/m) * ((X.T @ (h - y)) + lamda * theta0)
        theta = theta - alpha * gradient
    theta = theta.ravel()
    return (theta, J[-1], J)

def predict(theta, X):
    X = addOnes(X)
    h = sigmoid(X @ theta)
    threshold = 0.5
    predict = (h > threshold).astype(int)
    return predict

def accuracy(theta, X, y):
    p = predict(theta, X)
    compare = np.equal(p.reshape(1,-1),y.reshape(1,-1)).astype(int)
    accuracy = np.mean(compare)
    return accuracy
    

def plot2D(X, y):
    pos = np.where(y == 1)
    neg = np.where(y == 0)
    plt.scatter(X[pos, 0], X[pos, 1], marker='+', c='b')
    plt.scatter(X[neg, 0], X[neg, 1], marker='o', c='y')
    plt.show()
    
def plot2DLog(theta, X, y):
    pos = np.where(y == 1)
    neg = np.where(y == 0)
    plt.scatter(X[pos, 0], X[pos, 1], marker='+', c='b')
    plt.scatter(X[neg, 0], X[neg, 1], marker='o', c='y')
    x_lin = np.linspace(np.min(X[:,0]) , np.max(X[:,0]), 100)
    plt.plot(x_lin, - (x_lin * theta[1] + theta[0])/theta[2])
    plt.show()
    
def plotPolyMesh2DLog(X, y, p, C=1e5):
    poly = PolynomialFeatures(p)
    X_poly = poly.fit_transform(X)

    logreg = linear_model.LogisticRegression(C=C)
    logreg.fit(X_poly,y.ravel())

    theta3 = np.append(logreg.intercept_, logreg.coef_)
    
    pts = 200
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, pts), np.linspace(y_min, y_max, pts))
    Z = logreg.predict( poly.fit_transform(np.c_[xx.ravel(), yy.ravel()])[:,:]  )

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(1, figsize=(4, 4))
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)
    plt.xlabel('1')
    plt.ylabel('2')

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())

    plt.show()
    
def predictOneVsAll(all_theta, X):
    m = X.shape[0]
    num_labels = all_theta.shape[0]
    X = addOnes(X)
    predict = (X @ all_theta.T).argmax(1)
    return predict

def accuracyOneVsAll(all_theta, X, y):
    p = predictOneVsAll(all_theta, X)
    compare = np.equal(p.reshape(1,-1), y.reshape(1,-1)).astype(int)
    accuracy = np.mean(compare)  
    return accuracy

def predictNN(theta1, theta2, X):
    m = X.shape[0]
    num_labelsels = theta2.shape[0]
    
    Theta1 = theta1.T
    Theta2 = theta2.T
    
    a1 = X
    
    z1 = addOnes(a1) @ Theta1
    
    a2 = sigmoid(z1)
    
    z2 = addOnes(a2) @ Theta2
    
    a3 = sigmoid(z2)
    
    predict = a3.argmax(1)
    
    return predict

def accuracyNN(theta1, theta2, X, y):
    p = predictNN(theta1, theta2, X)
    compare = np.equal(p.reshape(1,-1), y.reshape(1,-1)).astype(int)
    accuracy = np.mean(compare)  
    return accuracy

def accuracyNN(theta1, theta2, X, y):
    p = predictNN(theta1, theta2, X)
    compare = np.equal(p.reshape(1,-1), y.reshape(1,-1)).astype(int)
    accuracy = np.mean(compare)  
    return accuracy

In [47]:
input_layer_size  = 400;
hidden_layer_size = 25; 
num_labels = 10; 

file = r'../data/ex3data1.mat'
print(scipy.io.whosmat(file))    # inspect what kinds of data the file has
mat = scipy.io.loadmat(file)
X = mat['X']; print(X.shape)
y = mat['y']; print(y.shape)
y = y - 1   # original y data ranges from 1 to 10.

file2 = r'../data/ex3weights.mat'
print(scipy.io.whosmat(file2))    # inspect what kinds of data the file has
mat2 = scipy.io.loadmat(file2)
theta1 = mat2['Theta1']
theta2 = mat2['Theta2']

print(np.unique(y))

[('X', (5000, 400), 'double'), ('y', (5000, 1), 'double')]
(5000, 400)
(5000, 1)
[('Theta1', (25, 401), 'double'), ('Theta2', (10, 26), 'double')]
[0 1 2 3 4 5 6 7 8 9]


In [48]:
print(accuracyNN(theta1, theta2, X, y))

0.9752


In [69]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(25), random_state=1)

clf.fit(X, y.ravel())

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=25, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [72]:
pred = clf.predict(X)
print(pred.shape)
print(np.unique(pred))

(5000,)
[0 1 2 3 4 5 6 7 8 9]


In [73]:
compare = np.equal(pred.reshape(1,-1), y.reshape(1,-1)).astype(int)

In [74]:
start = 4000
end = start + 100
print(pred[start:end])
print(y[start:end].ravel())

np.mean(compare)

[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


1.0